# Is there a corelation between the weather or the days of the week and how much I tend to watch on Netflix?

Here we are going to analyze my Netflix viewing history from a downloaded csv file that lists the title of what I watched and the date of when I watched it. Combingin this with local weather data from an API and the days of the week will allow me to test if:

* I tend to watch more on the weekends
* Watch more when the weather is poor (ie. raining heavily)

To test this I will do a hypothesis test of the frequency of things watched on Fridays, Saturdays, and Sundays ( ie. my classification of the weekend) compared to how much I watched in the week (ie. Monday-Thursday)
>NOTE: I am classifying Friday as the weekend because I believe I probably watched more in the evening on a Friday due to usually not having a responsiblity I need to wake up for the next morning

To do these test and combine these separate pieces of data. But, we first need to set up APIs to get the weather from certain days and the days of the week. 
Since the first instance from the NEtflix Viewing history csv file is dated "7/30/16" we need to import weather data and the days of the week from "7/30/16" to the last recorded day in the viewing hisotry file which is "4/9/2022".
>NOTE: for the weather I am going to just use the weather data from the Sammamish, WA area (where I live) since for the most part I watch netflix at home.

## Code to import all of the data
##### Importing the Weather
Getting the longitude and latitude for Sammamish.
This will allow me to use a spearate API to get the closest weather station ID to Sammamish

In [318]:
import json
import numpy as np
import pandas as pd
import requests

f = open("lat.txt", 'r')
key = f.read()
long_lat_url = "http://www.mapquestapi.com/geocoding/v1/address?key=" + key + "&location=Sammamish"
request_city = requests.get(long_lat_url)
json_city_str = request_city.text
json_city_obj = json.loads(json_city_str)

# getting latitiude and longititude of Sammamish
city_results_list = json_city_obj["results"]
dict = city_results_list[0]
locations_list = dict["locations"]
locations_dict = locations_list[0]
longLat_dict = locations_dict["displayLatLng"]
latitude = longLat_dict["lat"]
longitude = longLat_dict["lng"]

print("Sammamish latitude:", latitude)
print("Sammamish longitude:",longitude)

Sammamish latitude: 47.60553
Sammamish longitude: -122.035555


Getting the weather stations ID using a separate API:

In [319]:
f = open("Weather.txt", 'r')
key = f.read()
headers = {"x-rapidapi-key": key}
url = "https://meteostat.p.rapidapi.com/stations/nearby?lat=" + str(latitude) + "&lon=" + str(longitude) + "&limit=1"
request = requests.get(url = url, headers = headers)

json_station_str = request.text
json_station_obj = json.loads(json_station_str)
station_list = json_station_obj["data"]
station_dict = station_list[0]
station_id = station_dict["id"]
print("Weather Station ID:", station_id)

Weather Station ID: KRNT0


Now finally here we are loading the weather collected from this weather station from during the time period of 7/30/16 to 4/9/2022:

In [320]:
# get daily weather from station 2016-07-30 through 2022-04-09
headers = {"x-rapidapi-key": key}
weather_url = "https://meteostat.p.rapidapi.com/stations/daily?station="+ station_id + "&start=2016-07-30&end=2022-04-09"
request_weather = requests.get(url = weather_url, headers = headers)
json_weather_str = request_weather.text
json_weather_obj = json.loads(json_weather_str)
weather_df = pd.DataFrame(json_weather_obj["data"])
# weather_df.set_index("date", inplace = True)
weather_df.to_csv("daily_weather.csv")
print(weather_df)


            date  tavg  tmin  tmax  prcp  snow   wdir  wspd  wpgt    pres  \
0     2016-07-30  19.8  15.6  25.0   0.0  None    NaN  11.2  None  1014.1   
1     2016-07-31  18.4  14.4  22.8   0.0  None    NaN   9.1  None     NaN   
2     2016-08-01  18.0  12.8  25.0   0.0  None    NaN   6.9  None  1018.2   
3     2016-08-02  17.6  15.0  21.7   NaN  None    NaN   NaN  None     NaN   
4     2016-08-03  19.2  15.6  23.9   0.0  None    NaN   NaN  None  1024.2   
...          ...   ...   ...   ...   ...   ...    ...   ...   ...     ...   
2075  2022-04-05   7.6   4.0  12.0   0.8  None  172.0  14.7  None  1025.3   
2076  2022-04-06   8.8   2.0  16.0   0.0  None    6.0   9.7  None  1030.7   
2077  2022-04-07  14.3   7.0  22.0   0.0  None  354.0   7.7  None  1021.4   
2078  2022-04-08  10.1   7.0  15.0  10.2  None  194.0  18.2  None  1022.6   
2079  2022-04-09   6.4   3.0   9.0   6.4  None  159.0  12.1  None  1024.6   

      tsun  
0     None  
1     None  
2     None  
3     None  
4     None

##### Importing the days of the week:
Next we need to load in the days of the week from 2016-07-30 through to 2022-04-09.
To do this we are using the datetime library to create a dataframe with the dates within this parameter and then the corresponding day of the week


In [332]:

date_df = pd.DataFrame()
date_df["Date"] = pd.Series(pd.date_range('2016-7-30', '2022-4-9', freq='D'))
date_df["Day of Week"] = date_df['Date'].dt.day_name()
# change to string so I can compare easily

change_ser = pd.Series(list(string))

for value in range(len(date_df["Date"])):
    temp = str(date_df["Date"][value]).split(" ")
    string = temp[0]
    change_ser[value] = string
date_df["Date"] = change_ser

print(type(date_df["Date"][1]))
date_df.set_index("Date", inplace = True)
date_df.to_csv("Days_of_The_Week.csv")
print(date_df)

<class 'str'>
           Day of Week
Date                  
2016-07-30    Saturday
2016-07-31      Sunday
2016-08-01      Monday
2016-08-02     Tuesday
2016-08-03   Wednesday
...                ...
2022-04-05     Tuesday
2022-04-06   Wednesday
2022-04-07    Thursday
2022-04-08      Friday
2022-04-09    Saturday

[2080 rows x 1 columns]


## Cleaning the data
Now that we have loaded all of the data we need to clean the files in order to accurately compute data and make hypotheses

#### Cleaning the Weather DataFrame
First we are going to clean the weather dataframe.
Converting the temperatures to farenheit and then deleting the columns with more than 50 percent of teh values missing

In [322]:
import utils
# Cleaning and Interpolating
weather_df.replace("", np.NaN, inplace = True)
length_df = len(weather_df)
for value in weather_df:
    ser = weather_df[value].copy()
    for i in range(len(ser)):
        # if col is something you can interpolate
        if(value == "tavg", "tmin", "tmax", "prcp", "wdir", "wspd", "pres" and ser.isnull()):
            ser.interpolate(inplace = True)
    weather_df[value] = ser
    # removing columns with more than 50 percent of values missing
    count = weather_df[value].isnull().sum()
    if count > (length_df / 2):
        del weather_df[value]

# # converting to imperial
# avg_temp_ser = weather_df["tavg"].copy()
# min_temp_ser = weather_df["tmin"].copy()
# max_temp_ser = weather_df["tmax"].copy()
# for value in range(len(weather_df)):
#     avg_temp_ser[value] = utils.convert_temp(avg_temp_ser[value])
#     min_temp_ser[value] = utils.convert_temp(min_temp_ser[value])
#     max_temp_ser[value] = utils.convert_temp(max_temp_ser[value])
# weather_df["tavg"] = avg_temp_ser
# weather_df["tmin"] = min_temp_ser
# weather_df["tmax"] = max_temp_ser
weather_df = weather_df.rename({"date": "Date"}, axis = "columns")
print(weather_df)
weather_df.set_index("Date", inplace=True)
weather_df.to_csv("cleaned_daily_weather.csv")


            Date  tavg  tmin  tmax  prcp   wdir       wspd     pres
0     2016-07-30  19.8  15.6  25.0   0.0    NaN  11.200000  1014.10
1     2016-07-31  18.4  14.4  22.8   0.0    NaN   9.100000  1016.15
2     2016-08-01  18.0  12.8  25.0   0.0    NaN   6.900000  1018.20
3     2016-08-02  17.6  15.0  21.7   0.0    NaN   8.566667  1021.20
4     2016-08-03  19.2  15.6  23.9   0.0    NaN  10.233333  1024.20
...          ...   ...   ...   ...   ...    ...        ...      ...
2075  2022-04-05   7.6   4.0  12.0   0.8  172.0  14.700000  1025.30
2076  2022-04-06   8.8   2.0  16.0   0.0    6.0   9.700000  1030.70
2077  2022-04-07  14.3   7.0  22.0   0.0  354.0   7.700000  1021.40
2078  2022-04-08  10.1   7.0  15.0  10.2  194.0  18.200000  1022.60
2079  2022-04-09   6.4   3.0   9.0   6.4  159.0  12.100000  1024.60

[2080 rows x 8 columns]


#### Cleaning the Date Column of the Netflix Dataframe
This will be converting the dates in this colunn from the format `4/9/22` to `2022-04-09`.
This will allow us the combine the other two data sets easily.

In [323]:
netflix_df = pd.read_csv("NetflixViewingHistory_project.csv")

date_ser = netflix_df["Date"].copy()

for date in range(len(date_ser)):
    temp = date_ser[date].split("/")
    year = "20" + temp[2]
    month = temp[0]
    if(len(temp[0]) == 1):
        month = "0" + month
    day = temp[1]
    if(len(temp[1]) == 1):
        day = "0" + day
    date_ser[date] = year + "-" + month + "-" + day
netflix_df["Date"] = date_ser
netflix_df.to_csv("cleaned_Netflix_Viewing_History.csv")
print(netflix_df["Date"])

0       2022-04-09
1       2022-04-09
2       2022-04-08
3       2022-04-08
4       2022-04-05
           ...    
2468    2016-08-01
2469    2016-08-01
2470    2016-07-30
2471    2016-07-30
2472    2016-07-30
Name: Date, Length: 2473, dtype: object


#### Joining The Days Of The Week And The Weather For That Day:
We are doing this since each of these dataframes have keys of the date, which can only have one instance, and since it will make it easier to compare and visualize this data.

In [324]:
merged_weather_dayWeek_df = weather_df.merge(date_df, on = ["Date"], how= "inner")
print(merged_weather_dayWeek_df)

            tavg  tmin  tmax  prcp   wdir       wspd     pres Day of Week
Date                                                                     
2016-07-30  19.8  15.6  25.0   0.0    NaN  11.200000  1014.10    Saturday
2016-07-31  18.4  14.4  22.8   0.0    NaN   9.100000  1016.15      Sunday
2016-08-01  18.0  12.8  25.0   0.0    NaN   6.900000  1018.20      Monday
2016-08-02  17.6  15.0  21.7   0.0    NaN   8.566667  1021.20     Tuesday
2016-08-03  19.2  15.6  23.9   0.0    NaN  10.233333  1024.20   Wednesday
...          ...   ...   ...   ...    ...        ...      ...         ...
2022-04-05   7.6   4.0  12.0   0.8  172.0  14.700000  1025.30     Tuesday
2022-04-06   8.8   2.0  16.0   0.0    6.0   9.700000  1030.70   Wednesday
2022-04-07  14.3   7.0  22.0   0.0  354.0   7.700000  1021.40    Thursday
2022-04-08  10.1   7.0  15.0  10.2  194.0  18.200000  1022.60      Friday
2022-04-09   6.4   3.0   9.0   6.4  159.0  12.100000  1024.60    Saturday

[2080 rows x 8 columns]


In [350]:
freq_list = [0] * len(merged_weather_dayWeek_df)
freq_watched_ser = pd.Series(freq_list)

freq_df = pd.DataFrame(columns = ["Date", "Things Watched"])
freq_df["Things Watched"] = freq_watched_ser
freq_df["Date"] = date_df.index
freq_df["Things Watched"] = freq_watched_ser
freq_df.set_index("Date", inplace = True)
# print(freq_df)
freq_df.index = freq_df.index.astype(str)
# print(type(freq_df.index[1]))
# for date in netflix_df["Date"]:
#     freq_df[date]["Things Watched"] += 1

print(freq_df.index)
print(netflix_df["Date"])



Index(['2016-07-30', '2016-07-31', '2016-08-01', '2016-08-02', '2016-08-03',
       '2016-08-04', '2016-08-05', '2016-08-06', '2016-08-07', '2016-08-08',
       ...
       '2022-03-31', '2022-04-01', '2022-04-02', '2022-04-03', '2022-04-04',
       '2022-04-05', '2022-04-06', '2022-04-07', '2022-04-08', '2022-04-09'],
      dtype='object', name='Date', length=2080)
0       2022-04-09
1       2022-04-09
2       2022-04-08
3       2022-04-08
4       2022-04-05
           ...    
2468    2016-08-01
2469    2016-08-01
2470    2016-07-30
2471    2016-07-30
2472    2016-07-30
Name: Date, Length: 2473, dtype: object
